In [1]:
import cv2
import os
import pickle
import time
from finder import return_similar
from app.web_interface import get_engine, ENGINE_CHAIR, ENGINE_SOFA, ENGINE_POT

Using Theano backend.


Gallery yolo detections loaded!
Gallery CNN features loaded!
<_io.BufferedReader name='pickles/clock_vgg16.pickle'> loaded
<_io.BufferedReader name='pickles/bed_vgg16.pickle'> loaded
<_io.BufferedReader name='pickles/chair_vgg16.pickle'> loaded
<_io.BufferedReader name='pickles/plant_pot_vgg16.pickle'> loaded
<_io.BufferedReader name='pickles/sofa_vgg16.pickle'> loaded
<_io.BufferedReader name='pickles/table_vgg16.pickle'> loaded
All engines initiated


In [2]:
QUERY_IMAGES = './app/static/images/room_scenes/dev'
ALLOWED_CLASSES = ['chair']
ALL_CLASSES = ['chair', 'pottedplant', 'plant_pot', 'table', 'diningtable', 'sofa', 'mirror', 'pillow']
SEARCH_DIR = './app/static/images/sofa'
ENGINE = ENGINE_SOFA

In [3]:
def timeit(func):
    """Measure function running time"""
    def timed(*args, **kw):
        ts = time.time()
        result = func(*args, **kw)
        te = time.time()
        print('%s was completed in %2.2f sec' %
                     (func.__name__, te - ts))
        return result
    return timed

In [4]:
with open("pickles/room_to_items.p", "rb") as handle:
    room_items = {}
    pickle_data = pickle.load(handle)
    for (key, value) in pickle_data.items():
        room_items[os.path.basename(key)] = value

In [5]:
print (room_items["ikea-a-fresh-way-to-keep-your-clothes-tidy__1364315963128-s4.jpg"])

['chest of 3 drawers', 'mirror', 'pillow', 'quilt cover and 4 pillowcases', 'rug, high pile', 'cushion cover', 'work lamp', 'frame', 'box with compartments', 'handle']


In [11]:
with open("pickles/item_to_room.p", 'rb') as handle :
        ground_truth_data = {}
        room_objects = {}
        pickle_data = pickle.load(handle)
        for (key, value) in pickle_data.items():
            ground_truth_data[os.path.basename(key)] = value
            for word in ALL_CLASSES:
                if word in key:
                    for room in value:
                        if room == "ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg":
                            print(key)
                        try:
                            room_objects[os.path.basename(room)].append(word)
                        except:
                            room_objects[os.path.basename(room)] = []
                            room_objects[os.path.basename(room)].append(word)
#         print (ground_truth_data)

In [12]:
print (room_objects["ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg"])
print (ground_truth_data['702.922.98.jpg'])


KeyError: 'ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg'

Check if all images in dictionary

In [10]:
DIRECTORY = './app/static/images/room_scenes'
for image in os.listdir(DIRECTORY):
    try:
        val = room_objects[image]
    except KeyError:
        print(image)

.DS_Store
dev
ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg
ikea-a-fresh-green-kitchen-for-fresh-green-cooking__1364316022551-s4.jpg
ikea-a-fresh-look-for-a-first-bedroom__1364311992816-s4.jpg
ikea-a-fresh-way-to-sort-store-and-save-space__1364309607529-s4.jpg
ikea-a-grab-and-go-hallway-for-all-the-family__1364309608195-s4.jpg
ikea-a-small-living-room-ready-for-whiling-the-afternoon-away__1364309460486-s4.jpg
ikea-a-soft-pastel-place-to-nurture-and-grow__1364308476409-s4.jpg
ikea-a-traditional-kitchen-for-the-modern-life__1364308465964-s4.jpg
ikea-add-order-to-your-bedroom-with-the-brusali-wardrobe-in-white-from-ikea.-classic-style-at-an-afforable-price.-__1364309703765-s4.jpg
ikea-an-entry-as-organised-as-you-are__1364308425157-s4.jpg
ikea-bathroom-storage-__1364309108346-s4 2.jpg
ikea-big-world-small-scale__1364319297214-s4.jpg
ikea-big-world-small-scale__1364319297984-s4.jpg
ikea-bright-storage-for-the-smallest-of-spaces__1364316014982-s4 2.jpg
ikea-bring-a-to

Check what rooms have potted plants

Ground truth data is in format dictionary(product image) = scene image

In [10]:
def object_in_room(object_class, room):
    if object_class == "pottedplant":
        object_class = "plant_pot"
    try:
        objects = room_objects[os.path.basename(room)]
        print('Objects in this room', objects)
    except:
        print('Room is not in dictionary!', room)
        return False
    for object_item in objects:
        for word in object_item.split():
            if word == "table" and object_class == "diningtable":
                return True
            if word == object_class:
                print('Word = object class', word)
                return True
    print('object is not in the image')
    return False


def evaluation_test(query, result, ground_truth):
#     print ('Checking objects for room', query)
    for match in result:
        for room in ground_truth[match]:
#             print (match, ' is actually in room ', room)
            if os.path.basename(room) == query:
                return 1
    return 0

def calculate_accuracy(dataset, ground_truth, allowed_classes, k):
    nb_correct = [0] * (k+1)
    similar_images = [0] * (k+1)
    nb_all = 0
    nb_not_detected = 0
    for image_file in os.listdir(dataset):
        ndir = os.path.join(dataset, image_file)
        test_image = cv2.imread(ndir, cv2.IMREAD_GRAYSCALE)
        print(ndir)
        if test_image is not None:         
            for allowed_class in allowed_classes:
                if not object_in_room(allowed_class, ndir):
                    continue
                else:
                    search_dir, engine, static_path, object_class, object_image, object_image_path = get_engine(ndir, allowed_classes)
                    if object_class =="all":
                        print('Object of allowed class was NOT successfully detected')
                        nb_not_detected += 1
                        continue
                    nb_all += 1
                    for j in range(1, k+1):
                        similar_images = return_similar(object_image, search_dir, engine, j) 
                        res = evaluation_test(image_file, similar_images, ground_truth_data)
                        nb_correct[j] += res
                    for correct in nb_correct:
                        print ('Accuracy so far is ', correct/nb_all)
    print ("Not detected images :", nb_not_detected)
    for i in range(1, k+1):
        print ('Accuracy %i is %f' % (i, nb_correct[i]/nb_all))
        print('Nb all', nb_all)
    return [x/nb_all for x in nb_correct]
            
def accuracy_noyolo(dataset, ground_truth, allowed_classes, k, search_dir, engine):
    nb_correct = [0] * (k+1)
    similar_images = [0] * (k+1)
    nb_all = 0
    for image_file in os.listdir(dataset):
        ndir = os.path.join(dataset, image_file)
        test_image = cv2.imread(ndir, cv2.IMREAD_GRAYSCALE)
        print(ndir)
        if test_image is not None:         
            for allowed_class in allowed_classes:
                if not object_in_room(allowed_class, ndir):
                    continue
                else:          
                    nb_all += 1
                    for j in range(1, k+1):
                        similar_images = return_similar(ndir, search_dir, engine, j) 
                        res = evaluation_test(image_file, similar_images, ground_truth_data)
                        nb_correct[j] += res
                    for correct in nb_correct:
                        print ('Accuracy so far is ', correct/nb_all)
    for i in range(1, k+1):
        print ('Accuracy %i is %f' % (i, nb_correct[i]/nb_all))
    print('Nb all', nb_all)
    return [x/nb_all for x in nb_correct]
        

In [ ]:
calculate_accuracy(QUERY_IMAGES, ground_truth_data, ALLOWED_CLASSES, 30)

# Recall curve

In [21]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
plotly.tools.set_credentials_file(username='ivonat', api_key='Ig4jufJUA0txrZFBUMOV')

### Chair and sofa

In [22]:
N = 30
k = list(range(1, N+1))
vgg19_yolo_chair = [0.238806,
0.380597,
0.440299,
0.485075,
0.544776,
0.582090,
0.626866,
0.641791,
0.649254,
0.664179,
0.679104,
0.686567,
0.701493,
0.731343,
0.753731,
0.753731,
0.776119,
0.783582,
0.783582,
0.798507,
0.798507,
0.820896,
0.820896,
0.820896,
0.820896,
0.828358,
0.835821,
0.835821,
0.843284,
0.858209,]
vgg16_yolo_chair = [0.216418,
0.291045,
0.395522,
0.432836,
0.500000,
0.537313,
0.552239,
0.582090,
0.619403,
0.649254,
0.664179,
0.664179,
0.671642,
0.671642,
0.679104,
0.701493,
0.716418,
0.716418,
0.716418,
0.738806,
0.761194,
0.768657,
0.768657,
0.776119,
0.783582,
0.791045,
0.805970,
0.805970,
0.820896,
0.820896]
bovw_chair = [0.013986,
0.034965,
0.041958,
0.048951,
0.062937,
0.083916,
0.097902,
0.104895,
0.111888,
0.146853,
0.153846,
0.160839,
0.174825,
0.174825,
0.174825,
0.188811,
0.188811,
0.202797,
0.209790,
0.209790,
0.209790,
0.216783,
0.216783,
0.223776,
0.230769,
0.244755,
0.258741,
0.272727,
0.279720,
0.279720]
vgg19_chair = [
    0.062937,
0.111888,
0.139860,
0.174825,
0.202797,
0.223776,
0.251748,
0.272727,
0.286713,
0.314685,
0.335664,
0.349650,
0.377622,
0.391608,
0.419580,
0.433566,
0.440559,
0.454545,
0.454545,
0.454545,
0.468531,
0.510490,
0.531469,
0.538462,
0.552448,
0.566434,
0.573427,
0.580420,
0.587413,
0.601399,
]
vgg16_chair = [0.062937,
0.083916,
0.125874,
0.160839,
0.167832,
0.167832,
0.188811,
0.216783,
0.230769,
0.237762,
0.258741,
0.286713,
0.307692,
0.321678,
0.321678,
0.335664,
0.335664,
0.335664,
0.342657,
0.349650,
0.363636,
0.363636,
0.370629,
0.377622,
0.384615,
0.391608,
0.391608,
0.398601,
0.405594,
0.412587]

In [23]:
vgg19_yolo_sofa = [0.111111,
0.138889,
0.138889,
0.222222,
0.277778,
0.305556,
0.333333,
0.361111,
0.416667,
0.416667,
0.444444,
0.444444,
0.527778,
0.527778,
0.583333,
0.583333,
0.638889,
0.638889,
0.638889,
0.750000,
0.777778,
0.777778,
0.805556,
0.805556,
0.805556,
0.805556,
0.805556,
0.833333,
0.888889,
0.888889]
vgg16_yolo_sofa = [
0.083333,
0.111111,
0.138889,
0.250000,
0.305556,
0.361111,
0.416667,
0.416667,
0.444444,
0.444444,
0.527778,
0.583333,
0.611111,
0.694444,
0.694444,
0.694444,
0.694444,
0.722222,
0.722222,
0.777778,
0.777778,
0.777778,
0.777778,
0.777778,
0.777778,
0.777778,
0.805556,
0.805556,
0.805556,
0.833333]
bovw_sofa = [0.024390,
0.036585,
0.048780,
0.048780,
0.073171,
0.073171,
0.085366,
0.134146,
0.146341,
0.158537,
0.158537,
0.207317,
0.219512,
0.231707,
0.243902,
0.268293,
0.292683,
0.304878,
0.329268,
0.341463,
0.353659,
0.353659,
0.353659,
0.390244,
0.402439,
0.402439,
0.414634,
0.426829,
0.426829,
0.439024]
vgg19_sofa=[0.024390,
0.036585,
0.036585,
0.048780,
0.073171,
0.085366,
0.121951,
0.146341,
0.146341,
0.146341,
0.158537,
0.170732,
0.170732,
0.182927,
0.182927,
0.231707,
0.243902,
0.280488,
0.292683,
0.304878,
0.317073,
0.329268,
0.341463,
0.353659,
0.402439,
0.414634,
0.414634,
0.414634,
0.414634,
0.414634]
vgg16_sofa = [0.024390,
0.048780,
0.048780,
0.060976,
0.097561,
0.097561,
0.121951,
0.146341,
0.158537,
0.158537,
0.170732,
0.195122,
0.207317,
0.207317,
0.207317,
0.219512,
0.268293,
0.292683,
0.292683,
0.329268,
0.341463,
0.353659,
0.390244,
0.402439,
0.414634,
0.414634,
0.439024,
0.439024,
0.451220,
0.463415]

In [30]:
trace3 = go.Scatter(
    x = k,
    y = vgg19_yolo_sofa,
    name = 'VGG 19 + YOLO',
    mode='lines',
    line = dict(
        color = ('rgb(0,100,80)'),
        width = 1)
#     mode = 'lines'
)
trace4 = go.Scatter(
    x = k,
    y = vgg16_yolo_sofa,
    name = 'VGG 16 + YOLO',
    mode='lines',
    line = dict(
        color = ('rgb(0,176,246)'),
        width = 1)
#     mode = 'lines'
)
trace5 = go.Scatter(
    x = k,
    y = bovw_sofa,
    name = 'BOVW (baseline)',
    mode='lines',
    line = dict(
        color = ('rgb(231,107,243)'),
        width = 1)
#     mode = 'lines'
)
trace6 = go.Scatter(
    x = k,
    y = vgg19_sofa,
    name = 'VGG 19',
    mode='lines',
    line = dict(
        color = ('rgba(0,100,80,0.5)'),
        width = 1)
#     mode = 'lines'
)
trace7 = go.Scatter(
    x = k,
    y = vgg16_sofa,
    name = 'VGG 16',
    mode='lines',
    line = dict(
        color = ('rgba(0,176,246,0.5)'),
        width = 1)
#     mode = 'lines'
)

In [31]:
trace0 = go.Scatter(
    x = k,
    y = vgg19_yolo_chair,
    showlegend = False,
    mode='lines',
    line = dict(
        color = ('rgb(0,100,80)'),
        width = 1)
#     mode = 'lines'
)

trace1 = go.Scatter(
    x = k,
    y = vgg16_yolo_chair,
    showlegend = False,
    mode='lines',
    line = dict(
        color = ('rgb(0,176,246)'),
        width = 1)
#     mode = 'lines'
)
trace2 = go.Scatter(
    x = k,
    y = bovw_chair,
    showlegend = False,
    mode='lines',
    line = dict(
        color = ('rgb(231,107,243)'),
        width = 1)
#     mode = 'lines'
)
trace12 = go.Scatter(
    x = k,
    y = vgg19_chair,
    showlegend = False,
    mode='lines',
    line = dict(
        color = ('rgba(0,100,80, 0.5)'),
        width = 1)
#     mode = 'lines'
)
trace13 = go.Scatter(
    x = k,
    y = vgg16_chair,
    showlegend = False,
    mode='lines',
    line = dict(
        color = ('rgba(0,176,246,0.5)'),
        width = 1)
#     mode = 'lines'
)

fig = plotly.tools.make_subplots(rows=1, cols=2, subplot_titles=('Chair', 'Sofa'))
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace12, 1, 1)
fig.append_trace(trace13, 1, 1)
fig.append_trace(trace3, 1, 2)
fig.append_trace(trace4, 1, 2)
fig.append_trace(trace5, 1, 2)
fig.append_trace(trace6, 1, 2)
fig.append_trace(trace7, 1, 2)
fig['layout'].update(height=400, )
fig['layout']['xaxis1'].update(title='Top k')
fig['layout']['yaxis1'].update(title='Mean recall')
fig['layout']['xaxis2'].update(title='Top k')
py.iplot(fig, filename='recall_sofa_chair')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ivonat/0 or inside your plot.ly account where it is named 'recall_sofa_chair'


### Table

In [38]:
vgg19_yolo_table = [0.081081,
0.135135,
0.135135,
0.189189,
0.189189,
0.243243,
0.270270,
0.270270,
0.351351,
0.432432,
0.486486,
0.486486,
0.513514,
0.567568,
0.567568,
0.567568,
0.567568,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595,
0.594595]
bovw_table = [0.006579,
0.013158,
0.019737,
0.032895,
0.039474,
0.046053,
0.065789,
0.092105,
0.098684,
0.098684,
0.111842,
0.131579,
0.138158,
0.138158,
0.144737,
0.151316,
0.164474,
0.184211,
0.184211,
0.190789,
0.217105,
0.223684,
0.230263,
0.250000,
0.263158,
0.263158,
0.263158,
0.263158,
0.263158,
0.269737]


In [39]:
trace0 = go.Scatter(
    x = k,
    y = vgg19_yolo_table,
    name = 'VGG 19 + YOLO',
    mode='lines+markers',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4)
#     mode = 'lines'
)
trace2 = go.Scatter(
    x = k,
    y = bovw_table,
    name = 'BOVW',
    mode='lines+markers',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4)
#     mode = 'lines'
)
data = [trace0, trace2]

layout = dict(title = 'Object class - table',
              xaxis = dict(title = 'Top k'),
              yaxis = dict(title = 'Mean recall'),
              showlegend=True
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='chair-recall')

### Sofa

In [44]:
trace0 = go.Scatter(
    x = k,
    y = vgg_sofa,
    name = 'VGG 19 + YOLO',
    mode='lines+markers',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4)
#     mode = 'lines'
)
trace1 = go.Scatter(
    x = k,
    y = vgg16_sofa,
    name = 'VGG 16 + YOLO',
    mode='lines+markers',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4)
#     mode = 'lines'
)
trace2 = go.Scatter(
    x = k,
    y = bovw_sofa,
    name = 'BOVW',
    mode='lines+markers',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4)
#     mode = 'lines'
)
data = [trace0, trace1, trace2]

layout = dict(title = 'Object class - sofa',
              xaxis = dict(title = 'Top k'),
              yaxis = dict(title = 'Mean recall'),
              showlegend=True
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='chair-recall')